In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
!tar xf spark-2.4.5-bin-hadoop2.7.tgz
!pip install -q findspark
!pip install -q pyspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"
import findspark
import pyspark
from pyspark.sql import SparkSession
from pyspark import SparkContext

findspark.init("spark-2.4.5-bin-hadoop2.7")
sc = pyspark.SparkContext('local[*]')
spark = SparkSession.builder.appName('abc').getOrCreate()

     |████████████████████████████████| 218.4MB 61kB/s 
     |████████████████████████████████| 204kB 44.4MB/s 


# DataFrame

## wczytywanie danych do dataframe

In [0]:
people_df = spark.read.json('people.json') #wczytywanie json

wikiData = spark.read.parquet('wiki.parquet') #wczytywanie parquet

sales_data = spark.read.csv('sales_info.csv', inferSchema = True, header=True) # wczytywanie csv

wyswietlanie zawartosci dataframe

In [0]:
people_df.show() #pokazuje tekstowo

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [0]:
people_df.toPandas() # pobiera cale dane do sterownika (warto limitowac output)

,age,name
0,NaN,Michael
1,30.0,Andy
2,19.0,Justin


## Podstawowe transformacje w api dataframe

In [0]:
import pyspark.sql.functions as f # moduł potrzebny do pisania zapytań

Select

In [0]:
people_df.select(f.col('name')).toPandas()

,name
0,Michael
1,Andy
2,Justin


Filter

In [0]:
(people_df
 .filter(f.col('age') > 21)
 .limit(1)
 ).toPandas()

,age,name
0,30,Andy


groupBy

In [0]:
(people_df
 .groupBy(f.col('name')).count()
 .limit(1)
 ).toPandas()

,name,count
0,Michael,1


Sort

In [0]:
(people_df
 .sort(f.col('age').desc()) # albo f.col('age').asc()
 .limit(10)
 ).toPandas()

Sortowanie po wielu kolumnach

In [0]:
(people_df
 .sort([f.col('name') ,f.col('age').desc()]) # albo f.col('age').asc()
 .limit(10)
 ).toPandas()

Zmienianie nazwy kolumny

In [0]:
(people_df
 .withColumnRenamed('age', 'wiek')
 .limit(10)
 ).toPandas()

## Funkcje agregujące

max

In [0]:
(sales_data
 .groupBy(f.col('Company'))
 .max('Sales')
 .limit(10)
 ).toPandas()

,Company,max(Sales)
0,APPL,750.0
1,GOOG,340.0
2,FB,870.0
3,MSFT,600.0


min

In [0]:
(sales_data
 .groupBy(f.col('Company'))
 .min('Sales')
 .limit(10)
 ).toPandas()

,Company,min(Sales)
0,APPL,130.0
1,GOOG,20.0
2,FB,350.0
3,MSFT,43.0


Sum

In [0]:
(sales_data
 .groupBy(f.col('Company'))
 .sum('Sales')
 .limit(10)
 ).toPandas()

,Company,sum(Sales)
0,APPL,1480.0
1,GOOG,560.0
2,FB,1220.0
3,MSFT,767.0


agg: przyjmuje rozne argumenty, uniwersalna metoda do robienia agregacji

In [0]:
(sales_data
 .groupBy(f.col('Company'))
 .agg({'Sales': 'max'})
 .limit(10)
 ).toPandas()

,Company,max(Sales)
0,APPL,750.0
1,GOOG,340.0
2,FB,870.0
3,MSFT,600.0


alternatywne podejscie do agg (lepsze, przyjmuje wiele wartosci na raz, mozna nadawac nazwy kolumn itd)

In [0]:
(sales_data
 .groupBy(f.col('Company'))
 .agg(
      f.min('Sales').alias('min'),
      f.max('Sales') - f.min('Sales'),
      f.avg('Sales'),
      f.stddev('Sales'),
      f.stddev_pop('Sales'),
      f.countDistinct('Person').alias('personCount'),
      f.count('Person')
  )
 .withColumn('avg(Sales)', f.round(f.col('avg(Sales)'),2)) # zaokrąglanie wartosci w kolumnach
 .orderBy(f.col('min').desc())
 ).toPandas()

,Company,min,(max(Sales) - min(Sales)),avg(Sales),stddev_samp(Sales),stddev_pop(Sales),personCount,count(Person)
0,FB,350.0,520.0,610.00,367.695526,260.000000,2,2
1,APPL,130.0,620.0,370.00,268.824602,232.808935,4,4
2,MSFT,43.0,557.0,255.67,300.939086,245.715735,3,3
3,GOOG,20.0,320.0,186.67,160.416126,130.979218,3,3


In [0]:
(sales_data
 .groupBy(f.col('Company'))
 .agg(
      f.max('Sales'),
      f.min('Sales'),
      (f.max('Sales') - f.min('Sales')).alias('max - min'),
      f.avg('Sales'),
 )
 .limit(10)
 ).toPandas()

,Company,max(Sales),min(Sales),max - min,avg(Sales)
0,APPL,750.0,130.0,620.0,370.000000
1,GOOG,340.0,20.0,320.0,186.666667
2,FB,870.0,350.0,520.0,610.000000
3,MSFT,600.0,43.0,557.0,255.666667


Tworzenie widoków

In [0]:
people_df.createOrReplaceGlobalTempView('people') # tworze widok

In [0]:
spark.sql("select * from global_temp.people").toPandas()

,age,name
0,NaN,Michael
1,30.0,Andy
2,19.0,Justin


Tworzenie dataframe z RDD

In [0]:
from pyspark.sql import Row #importuje klase row
lines = sc.textFile('people.txt').map(lambda x: x.split(',')) # wczytuje plik, zamieniam na 'csv'
people = lines.map(lambda x: Row(name=x[0], age=int(x[1]))) # tworze RDD skladajacy sie z obiektow klasy Row
schemaPeople = spark.createDataFrame(people) # tworze dataframe z RDD skladajacego sie z obiektow klasy Row

In [0]:
schemaPeople.toPandas()

Tworzenie RDD z dataframe

In [0]:
teenNames = schemaPeople.rdd # zwraca obiek rdd bedacy ekwiwalentem dataframe z rdd

## Funkcje okienkowe

Tworzenie okna

In [0]:
from pyspark.sql.window import Window # importuje potrzebna biblioteke
import pyspark.sql.functions as f

data = \
  [("Thin", "Cell Phone", 6000),
  ("Normal", "Tablet", 1500),
  ("Mini", "Tablet", 5500),
  ("Ultra thin", "Cell Phone", 5500),
  ("Very thin", "Cell Phone", 6000),
  ("Big", "Tablet", 2500),
  ("Bendable", "Cell Phone", 3000),
  ("Foldable", "Cell Phone", 3000),
  ("Pro", "Tablet", 4500),
  ("Pro2", "Tablet", 6500)]

df = spark.createDataFrame(data, ['product', 'category', 'revenue']) # potrzebny dataframe

In [7]:
df.printSchema()

root
 |-- product: string (nullable = true)
 |-- category: string (nullable = true)
 |-- revenue: long (nullable = true)



In [0]:
# tworze okno

w_spec = Window.partitionBy(f.col('category')).orderBy(f.col('revenue').desc())

In [20]:
# wykorzystywanie okna metoda programistyczna

(df
 .select('*',
         (f.max(f.col('revenue')).over(w_spec)).alias('max in category')
         )
 .orderBy(f.col('revenue'))
 .limit(10)
 ).toPandas()

,product,category,revenue,max in category
0,Normal,Tablet,1500,6500
1,Big,Tablet,2500,6500
2,Bendable,Cell Phone,3000,6000
3,Foldable,Cell Phone,3000,6000
4,Pro,Tablet,4500,6500
5,Mini,Tablet,5500,6500
6,Ultra thin,Cell Phone,5500,6000
7,Thin,Cell Phone,6000,6000
8,Very thin,Cell Phone,6000,6000
9,Pro2,Tablet,6500,6500


In [33]:
# wykorzystywanie w spark sql
df = spark.read.csv('social_deltas.csv', header=True, inferSchema=True)
df.limit(5).toPandas()
df.createOrReplaceTempView('DF')
(spark.sql(
    """
    SELECT *, SUM(delta) OVER (PARTITION BY url, service ORDER BY ts) AS total FROM DF
    """)
).limit(5).toPandas()

,url,ts,service,delta,total
0,url1,2018-08-15 00:00:00,fb,5,5
1,url1,2018-08-15 00:05:00,fb,15,20
2,url1,2018-08-15 00:11:00,fb,11,31
3,url1,2018-08-15 00:18:00,fb,14,45
4,url1,2018-08-15 00:21:00,fb,14,59


In [39]:
# to samo ale programatycznie

(df
 .select('*',
         f.sum(f.col('delta')).over(Window.partitionBy([f.col('url'), f.col('service')]).orderBy(f.col('ts')))
         )
 .limit(5)
 ).toPandas()

,url,ts,service,delta,"sum(delta) OVER (PARTITION BY url, service ORDER BY ts ASC NULLS FIRST unspecifiedframe$())"
0,url1,2018-08-15 00:00:00,fb,5,5
1,url1,2018-08-15 00:05:00,fb,15,20
2,url1,2018-08-15 00:11:00,fb,11,31
3,url1,2018-08-15 00:18:00,fb,14,45
4,url1,2018-08-15 00:21:00,fb,14,59


funkcja .lag()

In [40]:
df = spark.read.csv('social_totals.csv', header=True, inferSchema=True)
df.createOrReplaceTempView('df')
df.limit(5).toPandas()

,url,ts,service,total
0,url1,2018-08-15 00:00:00,tw,1
1,url1,2018-08-15 00:05:00,tw,4
2,url1,2018-08-15 00:11:00,tw,5
3,url1,2018-08-15 00:18:00,tw,8
4,url1,2018-08-15 00:21:00,tw,12


In [43]:
# lag w spark.sql

(spark.sql(
    """
    SELECT *, LAG(total, 1, 0) OVER (PARTITION BY url, service ORDER BY ts) as prev FROM df
    """
)
 .withColumn('delta', f.col('total') - f.col('prev'))
 .limit(10)
).toPandas()

,url,ts,service,total,prev,delta
0,url1,2018-08-15 00:00:00,fb,5,0,5
1,url1,2018-08-15 00:05:00,fb,20,5,15
2,url1,2018-08-15 00:11:00,fb,31,20,11
3,url1,2018-08-15 00:18:00,fb,45,31,14
4,url1,2018-08-15 00:21:00,fb,59,45,14
5,url1,2018-08-15 00:30:00,fb,67,59,8
6,url1,2018-08-15 00:35:00,fb,110,67,43
7,url1,2018-08-15 00:38:00,fb,230,110,120
8,url1,2018-08-15 00:41:00,fb,360,230,130
9,url2,2018-08-15 00:00:00,tw,1,0,1


In [49]:
# lag programatycznie

(df
 .select(
     '*',
     f.lag('total', count=1, default=0).over(
         Window.partitionBy(
             [f.col('url'), f.col('service')]).orderBy(f.col('ts'))).alias('previous')
     )
 .withColumn('delta', f.col('total') - f.col('previous'))
 .limit(10)
 ).toPandas()

,url,ts,service,total,previous,delta
0,url1,2018-08-15 00:00:00,fb,5,0,5
1,url1,2018-08-15 00:05:00,fb,20,5,15
2,url1,2018-08-15 00:11:00,fb,31,20,11
3,url1,2018-08-15 00:18:00,fb,45,31,14
4,url1,2018-08-15 00:21:00,fb,59,45,14
5,url1,2018-08-15 00:30:00,fb,67,59,8
6,url1,2018-08-15 00:35:00,fb,110,67,43
7,url1,2018-08-15 00:38:00,fb,230,110,120
8,url1,2018-08-15 00:41:00,fb,360,230,130
9,url2,2018-08-15 00:00:00,tw,1,0,1


### funkcje rankingowe

#### row_number

|val|#|
|---|-|
|100|1|
|100|2|
|100|3|
|100|4|

#### rank

|val|#|
|---|-|
|100|1|
|100|1|
|100|1|
|100|2|


#### dense_rank

|val|#|
|---|-|
|100|1|
|100|1|
|100|1|
|100|4|

funkcja .rank()

In [51]:
# ładowanie potrzebnych danych
df = spark.read.csv('social_totals_agg.csv', header=True, inferSchema=True)
df.createOrReplaceTempView('df')
df.printSchema()

root
 |-- url: string (nullable = true)
 |-- service: string (nullable = true)
 |-- total: integer (nullable = true)



In [61]:
(df
 .select(
     "*",
     f.rank().over(Window.partitionBy([f.col('service')]).orderBy(f.col('total').desc())).alias('rank')
 )
 .filter(f.col('rank') == 1) # tutaj moge dawac warunki do wyszukiwania po rankingu
 .limit(20)
 ).toPandas()

,url,service,total,rank
0,url10,tw,260,1
1,url3,fb,500,1


## Schema

In [0]:
rdd = sc.textFile('people.txt').map(lambda x: x.split(',')) # wczytuje przykladowy plik
rdd = rdd.map(lambda x: (x[0], x[1].strip())) # czyszczenie danych
rdd.take(10)

[('Michael', '29'), ('Andy', '30'), ('Justin', '19')]

In [0]:
import pyspark.sql.types as t # importuje potrzebny modul

In [0]:
columns = ['name', 'age'] #tworze iterator zawierajacy nazwy kolumn
fields = [t.StructField(column, t.StringType(), nullable=True) for column in columns] # tworze liste Struct Types z okreslonym typem danych
schema = t.StructType(fields) # tworze obiekt StructType ze zmiennej fields

In [0]:
schemaPeople = spark.createDataFrame(people, schema) # tworze DF z okreslonym schema

In [0]:
schemaPeople.printSchema() # podgladam schema dla danego DF

root
 |-- name: string (nullable = true)
 |-- age: string (nullable = true)



Zmiana typu danych kolumny w DF

In [0]:
schemaPeople.withColumn('age', f.col('age').cast(t.IntegerType()))

UDF - user defined functions

In [0]:
from pyspark.sql.functions import udf
cutting_udf = udf(lambda x: x[:8], t.StringType())

## ZADANIA Z ZAJĘĆ

In [0]:
!unzip pagecounts.zip
pagecounts = sc.textFile('pagecounts')

In [0]:
enPages = pagecounts.filter(lambda x: 'en' == x.split(' ')[1])
enPages.take(10)

['20090505-000000 en ! 4 170494',
 '20090505-000000 en !!! 21 306957',
 '20090505-000000 en !!!Fuck_You!!! 9 87025',
 '20090505-000000 en !!!Fuck_You!!!_and_Then_Some 2 17960',
 '20090505-000000 en !!!_(album) 4 16665',
 '20090505-000000 en !Kung_people 8 75641',
 '20090505-000000 en !T.O.O.H.! 2 34121',
 '20090505-000000 en !important 161 987385',
 '20090505-000000 en $ 2 46050',
 '20090505-000000 en $1 11 54820']

In [0]:
(enPages
 .map(lambda x: x.split(' '))
 .map(lambda x: (x[0][:8], int(x[3])))
 .reduceByKey(lambda x,y: x+y)
 ).take(10)

[('20090505', 7076855), ('20090507', 6175726)]

In [0]:
(enPages
 .map(lambda x: x.split(' '))
 .map(lambda x: (x[2], int(x[3])))
 .reduceByKey(lambda x,y: x+y)
 .sortBy(lambda x: x[1], ascending=False)
 ).take(10)

[('404_error/', 1066734),
 ('Special:Search', 468159),
 ('Main_Page', 451126),
 ('Special:Random', 157000),
 ('Special:Randompage', 115757),
 ('Swine_influenza', 18160),
 ('Deadpool_(comics)', 15587),
 ('Scrubs_(TV_series)', 11390),
 ('Wiki', 10856),
 ('index.html', 9170)]

In [0]:
enPagesSplitted = enPages.map(lambda x: x.split(' '))
df = enPagesSplitted.toDF()

In [0]:
(df
 .withColumn('_4', f.col('_4').cast(t.IntegerType()))
 .select(f.col('_3'), f.col('_4'))
 .groupBy('_3')
 .sum('_4')
 .sort(f.col('sum(_4)').desc())
 .limit(3)
 ).toPandas()

In [0]:
# rozwiazanie za pomoca RDD
(df
 .rdd
 .map(lambda x: (x[0][:8], x[1], x[2], x[3], x[4]))
 .toDF()
 .withColumn('_4', f.col('_4').cast(t.IntegerType()))
 .select(f.col('_1'), f.col('_4'))
 .groupBy(f.col('_1'))
 .sum('_4')
 .withColumnRenamed('sum(_4)', 'sum')
 .withColumnRenamed('_1', 'date')
 .limit(3)
 ).toPandas()

,date,sum
0,20090505,7076855
1,20090507,6175726
